# Polarization in NH - 2020 Presidential Election

## Data Wrangling

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import ranksums

### Read in and Cleanup Election Data

In [2]:
# Let's read in our excel sheets which contain data on the votes per town for each county.  
# We have one sheet per county so let's make a list of counties
counties = ['Belknap', 'Carroll', 'Cheshire', 'Coos', 'Grafton', 
                'Hillsborough', 'Merrimack', 'Rockingham', 'Strafford', 'Sullivan']

sheet_names = [i for i in range(len(counties))]
data = pd.read_excel('president-2020.xls', header = 3, sheet_name=sheet_names)

In [3]:
# We now have a dictionary of dataframes.  Let's look at one.  Then we'll clean these
# up and put them into one large dataframe.
data[0]

,Belknap County,"Trump and Pence, r","Biden and Harris, d",Jorgensen and Cohen lib,,.1
0,Alton,2420.0,1438.0,45.0,NaN,NaN
1,Barnstead,1698.0,1150.0,74.0,NaN,NaN
2,Belmont,2365.0,1609.0,78.0,NaN,NaN
3,Center Harbor,407.0,379.0,6.0,NaN,NaN
4,Gilford,2748.0,2508.0,57.0,NaN,NaN
5,Gilmanton,1423.0,1020.0,35.0,NaN,NaN
6,Laconia Ward 1,976.0,773.0,23.0,NaN,NaN
7,Laconia Ward 2,665.0,585.0,20.0,NaN,NaN
8,Laconia Ward 3,617.0,824.0,22.0,NaN,NaN
9,Laconia Ward 4,682.0,672.0,26.0,NaN,NaN


In [4]:
# We're going to clean up each dataframe of the dictionary
# and create one dataframe

results = pd.DataFrame(columns=['Town','Trump','Biden','Other','County'])

for i in range(10):

    county_data = data[i].copy()
   
    # drop columns
    county_data.drop(columns = [' ', ' .1'], inplace=True)
    
    # rename columns
    county_data.columns = ['Town','Trump','Biden','Other']
    
    # Add a county column
    county_data['County'] = counties[i]

    # Drop all rows below the word "TOTALS"
    idx = np.where(county_data['Town']=='TOTALS')[0][0]
    county_data = county_data.iloc[:idx]
    
    results = pd.concat([results, county_data])

In [5]:
# Strip leading and trailing spaces from towns
results['Town'] = results['Town'].str.strip()

# Note that we have Wards for some towns so we need to group by those
results[results['Town'].str.find('Ward')>0]

,Town,Trump,Biden,Other,County
6,Laconia Ward 1,976.0,773.0,23.0,Belknap
7,Laconia Ward 2,665.0,585.0,20.0,Belknap
8,Laconia Ward 3,617.0,824.0,22.0,Belknap
9,Laconia Ward 4,682.0,672.0,26.0,Belknap
10,Laconia Ward 5,608.0,440.0,28.0,Belknap
...,...,...,...,...,...
24,Somersworth Ward 4,488.0,787.0,22.0,Strafford
25,Somersworth Ward 5,386.0,555.0,15.0,Strafford
2,Claremont Ward 1,781.0,819.0,50.0,Sullivan
3,Claremont Ward 2,1031.0,1172.0,48.0,Sullivan


In [6]:
def strip_ward(st):
    if st.find('Ward')>0:
        return st[:st.find('Ward')-1]
    else:
        return st
strip_ward('Nashua Ward 4')

'Nashua'

In [7]:
results['Town'] = results.apply(lambda x: strip_ward(x['Town']), axis=1)
results[results['Town'].str.find('Ward')>0]

,Town,Trump,Biden,Other,County


In [8]:
# Now group by Town, summing the votes
results = results.groupby(by=['Town', 'County']).sum()

In [9]:
results.reset_index(inplace=True)

In [10]:
results.set_index('Town', inplace=True)

In [11]:
# Let's drop towns with 0 vote counts because we want to look at a win ratio and that
# is undefined for no votes
results = results[~(results['Trump']==0)]
results = results[~(results['Biden']==0)]

In [12]:
results.shape

(238, 4)

In [13]:
# Let's just check that our lists of towns is unique
results.index.is_unique

True

In [14]:
# ok we're set to go.  Let's create some values of interest
results['Perc_Trump'] = results.apply(lambda x: x['Trump'] / (x['Trump'] + x['Biden']), axis=1)
results['Perc_Biden'] = results.apply(lambda x: x['Biden'] / (x['Trump'] + x['Biden']), axis=1)

In [15]:
results['Winner'] = results.apply(lambda x: 'Trump' if x['Perc_Trump'] > x['Perc_Biden'] else 'Biden', axis=1)

In [16]:
results.head()

,County,Trump,Biden,Other,Perc_Trump,Perc_Biden,Winner
Town,,,,,,,
Acworth,Sullivan,269.0,293.0,7.0,0.478648,0.521352,Biden
Albany,Carroll,190.0,250.0,9.0,0.431818,0.568182,Biden
Alexandria,Grafton,616.0,440.0,24.0,0.583333,0.416667,Trump
Allenstown,Merrimack,1233.0,1050.0,42.0,0.540079,0.459921,Trump
Alstead,Cheshire,477.0,684.0,28.0,0.410853,0.589147,Biden


In [17]:
results['wr_Biden'] = results.apply(lambda x: np.nan if x['Winner'] == 'Trump' else x['Biden']/x['Trump'], axis=1)
results['wr_Trump'] = results.apply(lambda x: np.nan if x['Winner'] == 'Biden' else x['Trump']/x['Biden'], axis=1)

In [18]:
results.head()

,County,Trump,Biden,Other,Perc_Trump,Perc_Biden,Winner,wr_Biden,wr_Trump
Town,,,,,,,,,
Acworth,Sullivan,269.0,293.0,7.0,0.478648,0.521352,Biden,1.089219,NaN
Albany,Carroll,190.0,250.0,9.0,0.431818,0.568182,Biden,1.315789,NaN
Alexandria,Grafton,616.0,440.0,24.0,0.583333,0.416667,Trump,NaN,1.400000
Allenstown,Merrimack,1233.0,1050.0,42.0,0.540079,0.459921,Trump,NaN,1.174286
Alstead,Cheshire,477.0,684.0,28.0,0.410853,0.589147,Biden,1.433962,NaN


### Import population and area data 

In [22]:
town_pop = pd.read_csv('Town Populations.csv', header=None)

In [23]:
town_pop.head()

,0,1,2
0,Manchester,"112,673",NaN
1,Nashua,"89,355",NaN
2,Concord,"43,627",NaN
3,Derry,"33,485",NaN
4,Dover,"32,191",NaN


In [24]:
town_pop = town_pop.drop(columns=2)

In [25]:
town_pop.columns=['Town','population']

In [26]:
# Strip leading and trailing spaces from towns
town_pop['Town'] = town_pop['Town'].str.strip()
# Clean up a few towns
town_pop['Town'] = town_pop.apply(lambda x: "Hart's Loc." if x['Town']=="Hart's Location" else x['Town'], axis=1)

town_pop['population'] = town_pop['population'].str.replace(',','')
town_pop['population'] = town_pop['population'].astype(int)

In [27]:
town_pop['population'] = town_pop['population'].astype(int)

In [28]:
town_pop.set_index('Town', inplace=True)
town_pop

,population
Town,
Manchester,112673
Nashua,89355
Concord,43627
Derry,33485
Dover,32191
...,...
Hart's Loc.,45
Millsfield,21
Dixville,11


In [29]:
# Read in area data
town_area = pd.read_csv('Town Area.csv')

In [30]:
# Strip leading and trailing spaces from towns
town_area['Town'] = town_area['Town'].str.strip()

# Clean up a few problem towns
town_area['Town'] = town_area.apply(lambda x: 'Moultonborough' if x['Town']=='Moultonboro' else x['Town'], axis=1)
town_area['Town'] = town_area.apply(lambda x: 'Waterville Valley' if x['Town']=='Waterville' else x['Town'], axis=1)
town_area['Town'] = town_area.apply(lambda x: "Wentworth's Loc." if x['Town']=="Wentworth's Location" else x['Town'], axis=1)
town_area['Town'] = town_area.apply(lambda x: "Hart's Loc." if x['Town']=="Hart's Location" else x['Town'], axis=1)

In [31]:
town_area.set_index('Town', inplace=True)
town_area

,Total Acres,Total Sq Miles,Land Acres,Land Sq Miles,Water Acres,Water Sq Miles
Town,,,,,,
Alton,"53,230.51",83.17,"40,906.19",63.92,"12,324.32",19.26
Barnstead,"28,758.63",44.94,"27,483.81",42.94,"1,274.82",1.99
Belmont,"20,427.47",31.92,"19,293.73",30.15,"1,133.73",1.77
Center Harbor,"10,394.43",16.24,"8,532.56",13.33,"1,861.86",2.91
Gilford,"34,243.66",53.51,"24,826.75",38.79,"9,416.90",14.71
...,...,...,...,...,...,...
Plainfield,"33,914.30",52.99,"33,417.72",52.22,496.58,0.78
Springfield,"28,478.79",44.50,"27,914.60",43.62,564.19,0.88
Sunapee,"16,099.13",25.15,"13,484.22",21.07,"2,614.92",4.09


### Import Household Income Data

In [32]:
town_income = pd.read_csv('Town Household Income.csv', index_col=0, header=None)

In [33]:
town_income.head()

,1,2
0,,
1,"Hampton Falls, New Hampshire","$129,531"
2,"Bedford, New Hampshire","$127,945"
3,"Amherst, New Hampshire","$127,246"
4,"Brookline, New Hampshire","$127,222"
5,"Hollis, New Hampshire","$126,379"


In [34]:
town_income.columns=['Town','Income']

In [35]:
town_income['Town'] = town_income['Town'].str.replace(', New Hampshire','')
# Strip leading and trailing spaces from towns
town_income['Town'] = town_income['Town'].str.strip()

In [36]:
town_income['Income'] = town_income['Income'].str.replace('$','')
town_income['Income'] = town_income['Income'].str.replace(',','')
town_income['Income'] = town_income['Income'].astype(int)

In [37]:
town_income

,Town,Income
0,,
1,Hampton Falls,129531
2,Bedford,127945
3,Amherst,127246
4,Brookline,127222
5,Hollis,126379
...,...,...
231,Littleton,39490
232,Clarksville,38382
233,Berlin,37969


In [38]:
town_income.set_index('Town', inplace=True)

In [39]:
town_income

,Income
Town,
Hampton Falls,129531
Bedford,127945
Amherst,127246
Brookline,127222
Hollis,126379
...,...
Littleton,39490
Clarksville,38382
Berlin,37969


### Import Zip Code Data

In [40]:
zips = pd.read_csv('zip codes.csv', index_col=None, header=None)

In [41]:
zips.head()

,0,1
0,3031,Amherst
1,3032,Auburn
2,3033,Brookline
3,3034,Candia
4,3036,Chester


In [42]:
zips.columns = ['zip_code', 'Town']

In [43]:
zips['zip_code'] = zips.apply(lambda x: '0' + str(x['zip_code']), axis=1)

In [44]:
zips.set_index('Town', inplace=True)

In [45]:
zips.head()

,zip_code
Town,
Amherst,03031
Auburn,03032
Brookline,03033
Candia,03034
Chester,03036


In [46]:
zips.index.is_unique

False

In [47]:
zips.shape

(353, 1)

We have multiple zip codes per town, so we need to do a right join when joining zip codes, and duplicate the values for those zip codes.  

### Join dataframes

In [48]:
# Join with population data
results = results.join(town_pop)

# Join with area data
results = results.join(town_area)

# Join with income data
results = results.join(town_income)

# Join with zip code data
results = results.join(zips, how='left')

In [49]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 284 entries, Acworth to Woodstock
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   County          284 non-null    object 
 1   Trump           284 non-null    float64
 2   Biden           284 non-null    float64
 3   Other           284 non-null    float64
 4   Perc_Trump      284 non-null    float64
 5   Perc_Biden      284 non-null    float64
 6   Winner          284 non-null    object 
 7   wr_Biden        140 non-null    float64
 8   wr_Trump        144 non-null    float64
 9   population      281 non-null    float64
 10  Total Acres     283 non-null    object 
 11  Total Sq Miles  283 non-null    float64
 12  Land Acres      283 non-null    object 
 13  Land Sq Miles   283 non-null    float64
 14  Water Acres     283 non-null    object 
 15  Water Sq Miles  283 non-null    float64
 16  Income          279 non-null    float64
 17  zip_code        283 non-null

In [50]:
results[results['population'].isnull()]

,County,Trump,Biden,Other,Perc_Trump,Perc_Biden,Winner,wr_Biden,wr_Trump,population,Total Acres,Total Sq Miles,Land Acres,Land Sq Miles,Water Acres,Water Sq Miles,Income,zip_code
Town,,,,,,,,,,,,,,,,,,
Hale's Location,Carroll,85.0,31.0,1.0,0.732759,0.267241,Trump,NaN,2.741935,NaN,"1,599.81",2.50,"1,599.77",2.50,0.04,0.00,67917.0,03860
Hart's Location,Carroll,20.0,23.0,0.0,0.465116,0.534884,Biden,1.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72000.0,03812
Wentworth's Loc.,Coos,13.0,4.0,1.0,0.764706,0.235294,Trump,NaN,3.250000,NaN,"12,326.13",19.26,"11,885.80",18.57,440.32,0.69,NaN,03579


In [51]:
results[results['Total Acres'].isnull()]

,County,Trump,Biden,Other,Perc_Trump,Perc_Biden,Winner,wr_Biden,wr_Trump,population,Total Acres,Total Sq Miles,Land Acres,Land Sq Miles,Water Acres,Water Sq Miles,Income,zip_code
Town,,,,,,,,,,,,,,,,,,
Hart's Location,Carroll,20.0,23.0,0.0,0.465116,0.534884,Biden,1.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72000.0,03812


In [52]:
results[results['Income'].isnull()]

,County,Trump,Biden,Other,Perc_Trump,Perc_Biden,Winner,wr_Biden,wr_Trump,population,Total Acres,Total Sq Miles,Land Acres,Land Sq Miles,Water Acres,Water Sq Miles,Income,zip_code
Town,,,,,,,,,,,,,,,,,,
Cambridge,Coos,4.0,3.0,0.0,0.571429,0.428571,Trump,NaN,1.333333,7.0,"33,098.74",51.72,"32,684.05",51.07,414.68,0.65,NaN,03588
Millsfield,Coos,16.0,5.0,0.0,0.761905,0.238095,Trump,NaN,3.200000,21.0,"28,937.79",45.22,"28,715.84",44.87,221.94,0.35,NaN,03579
Millsfield,Coos,16.0,5.0,0.0,0.761905,0.238095,Trump,NaN,3.200000,21.0,"28,937.79",45.22,"28,715.84",44.87,221.94,0.35,NaN,03582
Millsfield,Coos,16.0,5.0,0.0,0.761905,0.238095,Trump,NaN,3.200000,21.0,"28,937.79",45.22,"28,715.84",44.87,221.94,0.35,NaN,03588
Wentworth's Loc.,Coos,13.0,4.0,1.0,0.764706,0.235294,Trump,NaN,3.250000,NaN,"12,326.13",19.26,"11,885.80",18.57,440.32,0.69,NaN,03579


In [55]:
# I've checked and we really are missing this data so let's drop them. They are mostly for super small towns
results = results.dropna(subset=['population', 'Income','Total Acres'])

In [56]:
results.shape

(277, 18)

In [58]:
# Let's calculate population density as population / land area
results['pop_density'] = results['population'] / results['Land Sq Miles']

### Export Data

In [59]:
results.to_csv('results.csv')